TODO: 

add some plotting

add way to determine clustering linkage distances

clean up later modules

put cluster selection into module

make cluster selection work with sparse inputs

chase down memory usage in cluster selection

# Setup

In [ ]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

# check environment
import os
print(f'Conda Environment: ' + os.environ['CONDA_DEFAULT_ENV'])

from platform import python_version
print(f'python version: {python_version()}')

In [ ]:
import os
from pathlib import Path
# import scipy.io
# import scipy.sparse
import matplotlib.pyplot as plt
import numpy as np
import sklearn.manifold
import sklearn.cluster
import seaborn as sns
import pandas as pd
import sparse
import cv2
# import natsort

import torch
# from kymatio.torch import Scattering2D

import gc
from tqdm.notebook import tqdm
import copy
import time
import random
import functools
import multiprocessing as mp


In [ ]:
params = {
    'paths': {
        'dir_github': r'/media/rich/Home_Linux_partition/github_repos/',
        'dir_allOuterFolders': r"/media/rich/bigSSD/other lab data/Harnett_lab/ROI_Tracking/Vincent_Valerio/4th_email/AllStatFiles/rbp10",
        'pathSuffixToStat': 'plane1/stat.npy',
        'pathSuffixToOps': 'plane1/ops.npy',
    },
    'prefs': {
        # 'pref_use_NN_distances': True,
        'use_GPU': True,
        # 'pref_blurROIs': True,
    },    
    'importing': {
        'data_verbose': True,
        'out_height_width': [36, 36],
        'max_footprint_width': 1025,
        'type_meanImg': 'meanImgE',
        'images': None,
        'import_workers': -1,
    },
    'alignment': {
        'method': 'createOptFlow_DeepFlow',
        'kwargs_method': None,
        'return_sparse': True,
        'normalize': True,
    },
    'blurring': {
        'kernel_halfWidth': 2.0,
        'device': 'cpu',
        'plot_kernel': False,
        'batch_size': 2000,
    },
    'ROInet': {
        'device': 'cuda:0',
        'dir_networkFiles': '/home/rich/Downloads/ROInet',
        'download_from_gDrive': 'check_local_first',
        'gDriveID': '1FCcPZUuOR7xG-hdO6Ei6mx8YnKysVsa8',
        'verbose': True,
        'goal_size': 250,
        'ptile_norm': 90,
        'scale_norm': 0.6,
        'pref_plot': False,
        'batchSize_dataloader': 8,
        'pinMemory_dataloader': True,
        'persistentWorkers_dataloader': True,
        'prefetchFactor_dataloader': 2,
    },
    'SWT': {
        'kwargs_Scattering2D': {'J': 2, 'L': 8},
        'image_shape': (36, 36),
        'device': 'cuda:0',
    }, 
    'similarity': {
        'device': 'cpu',
        'n_workers': -1,
        'spatialFootprint_maskPower': 0.8,
        'block_height': 50,
        'block_width': 50,
        'overlapping_width_Multiplier': 0.1,
        'algorithm_nearestNeigbors_spatialFootprints': 'brute',
        'n_neighbors_nearestNeighbors_spatialFootprints': 'full',
        'locality': 1,
        'verbose': True,
    },
    'similarity_compute': {
        'linkage_methods': ['single', 'complete', 'ward', 'average'],
        # 'linkage_distances': math_functions.bounded_logspace(0.05, 2, 50),
        'min_cluster_size': 2,
        'max_cluster_size': None,
        'batch_size_hashing': 100,
        'cluster_similarity_reduction_intra': 'mean',
        'cluster_similarity_reduction_inter': 'max',
        'cluster_silhouette_reduction_intra': 'mean',
        'cluster_silhouette_reduction_inter': 'max',
        'n_workers': 100,
        'power_clusterSize': 2,
        'power_clusterSilhouette': 1.5,
    },
    'clusterAssigner': {
        'optimizer_partial_lr': 1e-1,
        'optimizer_partial_betas': (0.9, 0.900),
        'scheduler_partial_base_lr': 1e-3,
        'scheduler_partial_max_lr': 3e0,
        'scheduler_partial_step_size_up': 250,
        'scheduler_partial_cycle_momentum': False,
        'scheduler_partial_verbose': False,
        'dmCEL_temp': 1,
        'dmCEL_sigSlope': 2,
        'dmCEL_sigCenter': 0.5,
        'dmCEL_penalty': 1e0,
        'sampleWeight_softplusKwargs': {'beta': 500, 'threshold': 50},
        'sampleWeight_penalty': 1e2,
        'fracWeighted_goalFrac': 1.0,
        'fracWeighted_sigSlope': 2,
        'fracWeighted_sigCenter': 0.5,
        'fracWeight_penalty': 1e2,
        'maskL1_penalty': 2e-5,
        'tol_convergence': 1e-9,
        'window_convergence': 50,
        'freqCheck_convergence': 50,
        'verbose': True,       
    },
    'clusterAssigner_fit': {
        'min_iter': 1e3,
        'max_iter': 2e3,
        'verbose': True,
        'verbose_interval': 10,
        'm_threshold': 0.9,
    },
    'visualization': {
        'FOV_threshold_confidence': 0.5,
    }
}

In [ ]:
dir_github = Path(params['paths']['dir_github']).resolve()

import sys
sys.path.append(str(dir_github))

%load_ext autoreload
%autoreload 2
from basic_neural_processing_modules import path_helpers, torch_helpers, featurization, indexing, misc, torch_helpers, plotting_helpers, classification, math_functions, parallel_helpers, clustering, image_processing, pickle_helpers, container_helpers
from Big_Ugly_ROI_Tracker.multiEps.multiEps_modules import *
from registration_rClust import data_importing, visualization, alignment, blurring, helpers, ROInet, scatteringWaveletTransformer, similarity_graph, cluster_assignment


device = torch_helpers.set_device(use_GPU=params['prefs']['use_GPU'])

%matplotlib notebook


# Import paths
def print_list(l):
    for item in l:
        print(item)
        
dir_allOuterFolders = Path(params['paths']['dir_allOuterFolders']).resolve()

folders_allSessions = natsort.natsorted(path_helpers.get_dir_contents(dir_allOuterFolders)[0])

folders_toUse = folders_allSessions
# folders_toUse = list(map(folders_allSessions.__getitem__, [np.arange(1,9, dtype=np.int32)]))


# dir_allS2pFolders = [dir_allOuterFolders / folder / 'exp' / 'suite2p' / 'plane0' for folder in folders_toUse]
dir_allS2pFolders = [dir_allOuterFolders / folder for folder in folders_toUse]

pathSuffixToStat = params['paths']['pathSuffixToStat']
pathSuffixToOps = params['paths']['pathSuffixToOps']

paths_allStat = np.array([path / pathSuffixToStat for path in dir_allS2pFolders])[:]
paths_allOps  = np.array([path / pathSuffixToOps for path in dir_allS2pFolders])[:]

print(folders_allSessions)
print(folders_toUse)
print_list(dir_allS2pFolders)
print_list(paths_allStat)


#Import data
data = data_importing.Data_suite2p(
    paths_statFiles=paths_allStat,
    paths_opsFiles=paths_allOps,
    verbose=params['importing']['data_verbose'],
);

data.import_statFiles();

data.import_ROI_centeredImages(
    out_height_width=params['importing']['out_height_width'],
    max_footprint_width=params['importing']['max_footprint_width'],
);

data.import_FOV_images(
    type_meanImg=params['importing']['type_meanImg'],
    images=params['importing']['images'],
);

data.import_ROI_spatialFootprints(workers=params['importing']['import_workers']);

# visualization.display_toggle_image_stack(data.FOV_images)


# Alignment
aligner = alignment.Alinger(
    method=params['alignment']['method'],
    kwargs_method=params['alignment']['kwargs_method'],
)

aligner.register_ROIs(
    templateFOV=data.FOV_images[0],
    FOVs=data.FOV_images,
    ROIs=data.spatialFootprints,
    return_sparse=params['alignment']['return_sparse'],
    normalize=params['alignment']['normalize'],
);

# visualization.display_toggle_image_stack(aligner.FOVs_aligned)
# visualization.display_toggle_image_stack(aligner.get_ROIsAligned_maxIntensityProjection())


# Blur ROIs (optional)
blurrer = blurring.ROI_Blurrer(
    frame_shape=(data.FOV_height, data.FOV_width),
    kernel_halfWidth=params['blurring']['kernel_halfWidth'],
    device=params['blurring']['device'],
    plot_kernel=params['blurring']['plot_kernel'],
)

blurrer.blur_ROIs(
    spatialFootprints=aligner.ROIs_aligned,
    batch_size=params['blurring']['batch_size'],
);

# visualization.display_toggle_image_stack(blurrer.get_ROIsBlurred_maxIntensityProjection())


# Neural network embedding distances
hash_dict_true = {
    'params': ('params.json', '877e17df8fa511a03bc99cd507a54403'),
    'model': ('model.py', '6ef5c29793ae16a64e43e8cab33d9ff4'),
    'state_dict': ('ConvNext_tiny__1_0_unfrozen__simCLR.pth', 'a5fae4c9ea95f2c78b4690222b2928a5'),
}

roinet = ROInet.ROInet_embedder(
    device=params['ROInet']['device'],
    dir_networkFiles=params['ROInet']['dir_networkFiles'],
    download_from_gDrive=params['ROInet']['download_from_gDrive'],
    gDriveID=params['ROInet']['gDriveID'],
    hash_dict_networkFiles=hash_dict_true,
    verbose=params['ROInet']['verbose'],
)

roinet.generate_dataloader(
    ROI_images=data.ROI_images,
    goal_size=params['ROInet']['goal_size'],
    ptile_norm=params['ROInet']['ptile_norm'],
    scale_norm=params['ROInet']['scale_norm'],
    pref_plot=params['ROInet']['pref_plot'],
    batchSize_dataloader=params['ROInet']['batchSize_dataloader'],
    pinMemory_dataloader=params['ROInet']['pinMemory_dataloader'],
    numWorkers_dataloader=mp.cpu_count(),
    persistentWorkers_dataloader=params['ROInet']['persistentWorkers_dataloader'],
    prefetchFactor_dataloader=params['ROInet']['prefetchFactor_dataloader'],    
)

roinet.generate_latents();

gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()


# Scattering wavelet embedding distances
swt = scatteringWaveletTransformer.SWT(
    kwargs_Scattering2D=params['SWT']['kwargs_Scattering2D'], 
    image_shape=params['SWT']['image_shape'], 
    device=params['SWT']['device'],
)

swt.transform(ROI_images=np.concatenate(data.ROI_images, axis=0));


# Compute similarities
sim = similarity_graph.ROI_graph(
    device=params['similarity']['device'],
    n_workers=params['similarity']['n_workers'],
    spatialFootprint_maskPower=params['similarity']['spatialFootprint_maskPower'],
    frame_height=data.FOV_height,
    frame_width=data.FOV_width,
    block_height=params['similarity']['block_height'],
    block_width=params['similarity']['block_width'],
    overlapping_width_Multiplier=params['similarity']['overlapping_width_Multiplier'],
    algorithm_nearestNeigbors_spatialFootprints=params['similarity']['algorithm_nearestNeigbors_spatialFootprints'],
    n_neighbors_nearestNeighbors_spatialFootprints=params['similarity']['n_neighbors_nearestNeighbors_spatialFootprints'],
    locality=params['similarity']['locality'],
    verbose=params['similarity']['verbose'],
)

sim.visualize_blocks()

sim.compute_similarity_blockwise(
    spatialFootprints=blurrer.ROIs_blurred,
    features_NN=roinet.latents,
    features_SWT=swt.latents,
    ROI_session_bool=data.sessionID_concat,
    linkage_methods=params['similarity_compute']['linkage_methods'],
    linkage_distances=math_functions.bounded_logspace(0.05, 2, 50),
    min_cluster_size=params['similarity_compute']['min_cluster_size'],
    max_cluster_size=params['similarity_compute']['max_cluster_size'],
    batch_size_hashing=params['similarity_compute']['batch_size_hashing'],
);

sim.compute_cluster_similarity_graph(
        cluster_similarity_reduction_intra=params['similarity_compute']['cluster_similarity_reduction_intra'],
        cluster_similarity_reduction_inter=params['similarity_compute']['cluster_similarity_reduction_inter'],
        cluster_silhouette_reduction_intra=params['similarity_compute']['cluster_silhouette_reduction_intra'],
        cluster_silhouette_reduction_inter=params['similarity_compute']['cluster_silhouette_reduction_inter'],
        n_workers=params['similarity_compute']['n_workers'],
);

sim.compute_cluster_scores(
    power_clusterSize=params['similarity_compute']['power_clusterSize'], 
    power_clusterSilhouette=params['similarity_compute']['power_clusterSilhouette'],
);

# w_raw = torch.as_tensor((np.array(sim.cluster_bool.sum(1)).squeeze()**2) * (10**(sim.c_sil*1.5)))

# w_raw = torch.as_tensor(10**(np.array(sim.cluster_bool.sum(1)).squeeze()) * (10**(sim.c_sil*1)))

fig, axs = plt.subplots(1,2, figsize=(10,5))
axs[0].plot(w_raw.cpu())
# plt.ylim([0,1.1])
axs[1].plot(w_raw.cpu())
axs[1].set_yscale('log')

plt.figure()
plt.hist(w_raw.cpu(), 500)
plt.yscale('log')
plt.xscale('log')

plt.figure()
plt.scatter((np.array(sim.cluster_bool.sum(1)).squeeze()**2), w_raw, alpha=0.01)

clusterAssigner= cluster_assignment.Cluster_Assigner(
    c=sim.c_sim,
    h=sim.cluster_bool.T,
    w=sim.scores,
    device=DEVICE,
    m_init=(torch.ones(c.shape[0])*-5 + torch.rand(c.shape[0])*1).type(torch.float32),
    optimizer_partial=functools.partial(torch.optim.Adam, lr=params['clusterAssigner']['optimizer_partial_lr'], betas=params['clusterAssigner']['optimizer_partial_betas']),
    scheduler_partial=functools.partial(torch.optim.lr_scheduler.CyclicLR, base_lr=params['clusterAssigner']['scheduler_partial_base_lr'], max_lr=params['clusterAssigner']['scheduler_partial_max_lr'], step_size_up=params['clusterAssigner']['scheduler_partial_step_size_up'], cycle_momentum=params['clusterAssigner']['scheduler_partial_cycle_momentum'], verbose=params['clusterAssigner']['scheduler_partial_verbose']),
    dmCEL_temp=params['clusterAssigner']['dmCEL_temp'],
    dmCEL_sigSlope=params['clusterAssigner']['dmCEL_sigSlope'],
    dmCEL_sigCenter=params['clusterAssigner']['dmCEL_sigCenter'],
    dmCEL_penalty=params['clusterAssigner']['dmCEL_penalty'],
    sampleWeight_softplusKwargs=params['clusterAssigner']['sampleWeight_softplusKwargs'],
    sampleWeight_penalty=params['clusterAssigner']['sampleWeight_penalty'],
    fracWeighted_goalFrac=params['clusterAssigner']['fracWeighted_goalFrac'],
    fracWeighted_sigSlope=params['clusterAssigner']['fracWeighted_sigSlope'],
    fracWeighted_sigCenter=params['clusterAssigner']['fracWeighted_sigCenter'],
    fracWeight_penalty=params['clusterAssigner']['fracWeight_penalty'],
    maskL1_penalty=params['clusterAssigner']['maskL1_penalty'],
    tol_convergence=params['clusterAssigner']['tol_convergence'],
    window_convergence=params['clusterAssigner']['window_convergence'],
    freqCheck_convergence=params['clusterAssigner']['freqCheck_convergence'],
    verbose=params['clusterAssigner']['verbose'],
)

clusterAssigner.fit(
    min_iter=params['clusterAssigner_fit']['min_iter'],
    max_iter=params['clusterAssigner_fit']['max_iter'],
    verbose=params['clusterAssigner_fit']['verbose'], 
    verbose_interval=params['clusterAssigner_fit']['verbose_interval'],
)

clusterAssigner.plot_loss()

del clusterAssigner

gc.collect()

torch.cuda.empty_cache()

gc.collect()

torch.cuda.empty_cache()

clusterAssigner.plot_c_masked_matrix()
clusterAssigner.plot_clusterWeights()
clusterAssigner.plot_sampleWeights()

clusterAssigner.plot_clusterScores(bins=200)
plt.xscale('log')
plt.yscale('log')

preds, confidence, scores_samples, m_bool = clusterAssigner.predict(m_threshold=params['clusterAssigner_fit']['m_threshold'])

fig, axs = clusterAssigner.plot_labelCounts()
axs[0].set_ylim([0,20])

print(f'Number of clusters: {m_bool.sum()}')


# visualization
FOV_clusters = visualization.compute_colored_FOV(
    spatialFootprints=aligner.ROIs_aligned,
    FOV_height=data.FOV_height,
    FOV_width=data.FOV_width,
    preds=clusterAssigner.preds,
    confidence=clusterAssigner.confidence,
    threshold_confidence = params['visualization']['FOV_threshold_confidence'],
)

visualization.display_toggle_image_stack(FOV_clusters)